# No, Python is Not Too Slow for Computational Economics

### John Stachurski

#### Australian National University
john.stachurski@anu.edu.au

In [4]:
!date

Fri Sep 28 10:08:19 EDT 2018


Acknowledgements: Thanks to Quentin Batista and Natasha Watkins for helpful comments and suggestions.

## Introduction

In [this paper](https://github.com/jstac/julia_python_comparison/blob/master/Update_March_23_2018.pdf), S. Boragan Aruoba and Jesus Fernandez-Villaverde (AFV) study the relative speed of a range of programming lanugages by testing them on value function iteration.  In the abstract they state:

**"The central conclusions of our original paper remain unaltered: C++ is the fastest alternative, Julia offers a great balance of speed and ease of use, and Python is too slow."**

The last claim is strange, since the authors' own data do not support their conclusion.  In their table 1, Python combined with its scientific libraries is actually reported as slightly faster than Julia (2.31 seconds for Python vs 2.35 for Julia).

We rerun the code below and find a similar outcome: The Python code runs between 10% and 30% faster.  (Measurements vary depending on whether or not we include compile time and so on.  Further discussion of timing is given below.) The execution environment is 

```
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               142
Model name:          Intel(R) Core(TM) i5-7300U CPU @ 2.60GHz
```

We are using the latest version of each language and scientific ecosystem at the time of writing: Julia 1.0 and Anaconda Python 5.2.

Of course, we are comparing only Julia vs Python plus its scientific libraries (as opposed to Julia vs pure Python), since no scientist would consider foregoing the scientific libraries when implementing computationally intensive algorithms in Python.

Here "Python plus its scientific libraries" means the computing environment contained in the popular Anaconda Python distribution.



## Additional Comments in Favor of Python

1. Python is massively popular for both general purpose and scientific computing (see, e.g., [this ranking](https://spectrum.ieee.org/static/interactive-the-top-programming-languages-2018)).  Its popularity is partly due to (and partly caused by) the huge range of third party libraries.  Within the scientific domain, these libraries include high quality array processing software and a powerful JIT compiler.  If Python is "too slow" for scientific computing in economics, why is that not the case in, say, astrophysics or machine learning?

2. There has been some discussion in the past about making parts of Julia interpreted rather than compiled, since JIT compilation makes debugging harder and adds other complications.  Moreover, if one considers, say, a large plotting library, then very little of the code base will be critical to performance.  So why compile it?  In a sense, compiling the entire code base is a form of premature optimization --- decried as the "root of all evil" by [Donald Knuth](https://en.wikipedia.org/wiki/Donald_Knuth), one of the greats of scientific computing.  In contrast, Python is interpreted and JIT compilation is judiciously applied to small parts of one's code base.  Perhaps this is the sweet spot for scientific computing that Python has somehow stumbled into?

3. AFV do not display any profiling of their codes in the different languages. No memory footprint, no flop rates, etc. are shown. This matters because memory access patterns can always be optimized, especially in small problems as treated here.  One would have to take into account all the individual features of the programming languages to make a fair comparison. (Most likely there would be no surprise, as their performance behavior is known.)  We follow AFV in simply reporting timings but a useful reference for the topics listed above is: S. Goedecker and A. Hoisie. Performance Optimization of Numerically Intensive Codes (SIAM), 2001. ISBN 978-0898714845. 

## Additional Comments in Favor of Julia

1. All of Julia and its scientific libraries can be JIT compiled.  The same is not true of Python and its scientific libraries (although progress towards this goal has been rapid).

2. As an aside, the author thinks Julia is a great project and hopes that it succeeds.  Python wouldn't be where it is now without the flow of ideas (and competitive pressure) from Julia.

3. Python's primary scientific JIT compiler (Numba) and Julia's JIT compiler are built on the same architecture (LLVM).  Similar execution times are to be expected.  From this perspective it is surprising that the Python code runs faster on this machine.  Other users might experience different outcomes.

## Timings

The code below is exactly as in the [GitHub](https://github.com/jesusfv/Comparison-Programming-Languages-Economics) repo, except for the following changes, consistent with comments made in the 2018 paper. 

1. Doubled the length of the capital grid.
2. Convergence tolerance is 1e-8.
3. Print out status after every 20 iterations.
4. Changed print statements to print functions (Python 3 compliance)

In [5]:
# Basic RBC model with full depreciation (Alternate 1)
#
# Jesus Fernandez-Villaverde
# Haverford, July 3, 2013

import numpy as np
import math
import time
from numba import jit

# - Start Inner Loop - #
# - bbeta                   float
# - nGridCapital:           int64
# - gridCapitalNextPeriod:  int64
# - mOutput:                float (17820 x 5)
# - nProductivity:          int64
# - vGridCapital:           float (17820, )
# - mValueFunction:         float (17820 x 5)
# - mPolicyFunction:        float (17820 x 5)

@jit
def innerloop(bbeta, nGridCapital, gridCapitalNextPeriod, mOutput, nProductivity, vGridCapital, expectedValueFunction, mValueFunction, mValueFunctionNew, mPolicyFunction):

    for nCapital in range(nGridCapital):
        valueHighSoFar = -100000.0
        capitalChoice  = vGridCapital[0]
        
        for nCapitalNextPeriod in range(gridCapitalNextPeriod, nGridCapital):
            consumption = mOutput[nCapital,nProductivity] - vGridCapital[nCapitalNextPeriod]
            valueProvisional = (1-bbeta)*np.log(consumption)+bbeta*expectedValueFunction[nCapitalNextPeriod,nProductivity];

            if  valueProvisional > valueHighSoFar:
                valueHighSoFar = valueProvisional
                capitalChoice = vGridCapital[nCapitalNextPeriod]
                gridCapitalNextPeriod = nCapitalNextPeriod
            else:
                break 

        mValueFunctionNew[nCapital,nProductivity] = valueHighSoFar
        mPolicyFunction[nCapital,nProductivity]   = capitalChoice

    return mValueFunctionNew, mPolicyFunction

def main_func():

    #  1. Calibration

    aalpha = 1.0/3.0     # Elasticity of output w.r.t. capital
    bbeta  = 0.95        # Discount factor

    # Productivity values
    vProductivity = np.array([0.9792, 0.9896, 1.0000, 1.0106, 1.0212],float)

    # Transition matrix
    mTransition   = np.array([[0.9727, 0.0273, 0.0000, 0.0000, 0.0000],
                     [0.0041, 0.9806, 0.0153, 0.0000, 0.0000],
                     [0.0000, 0.0082, 0.9837, 0.0082, 0.0000],
                     [0.0000, 0.0000, 0.0153, 0.9806, 0.0041],
                     [0.0000, 0.0000, 0.0000, 0.0273, 0.9727]],float)

    ## 2. Steady State

    capitalSteadyState     = (aalpha*bbeta)**(1/(1-aalpha))
    outputSteadyState      = capitalSteadyState**aalpha
    consumptionSteadyState = outputSteadyState-capitalSteadyState

    print("Output = ", outputSteadyState, " Capital = ", capitalSteadyState, " Consumption = ", consumptionSteadyState) 

    # We generate the grid of capital
    vGridCapital           = np.arange(0.5*capitalSteadyState,1.5*capitalSteadyState,0.000005)

    nGridCapital           = len(vGridCapital)
    nGridProductivity      = len(vProductivity)

    ## 3. Required matrices and vectors

    mOutput           = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mValueFunction    = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mValueFunctionNew = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    mPolicyFunction   = np.zeros((nGridCapital,nGridProductivity),dtype=float)
    expectedValueFunction = np.zeros((nGridCapital,nGridProductivity),dtype=float)

    # 4. We pre-build output for each point in the grid

    for nProductivity in range(nGridProductivity):
        mOutput[:,nProductivity] = vProductivity[nProductivity]*(vGridCapital**aalpha)

    ## 5. Main iteration

    maxDifference = 10.0
    tolerance = 0.00000001
    iteration = 0

    log = math.log
    zeros = np.zeros
    dot = np.dot

    while(maxDifference > tolerance):

        expectedValueFunction = dot(mValueFunction,mTransition.T)

        for nProductivity in range(nGridProductivity):

            # We start from previous choice (monotonicity of policy function)
            gridCapitalNextPeriod = 0

            # - Start Inner Loop - #

            mValueFunctionNew, mPolicyFunction = innerloop(bbeta, nGridCapital, gridCapitalNextPeriod, mOutput, nProductivity, vGridCapital, expectedValueFunction, mValueFunction, mValueFunctionNew, mPolicyFunction)

            # - End Inner Loop - #

        maxDifference = (abs(mValueFunctionNew-mValueFunction)).max()

        mValueFunction    = mValueFunctionNew
        mValueFunctionNew = zeros((nGridCapital,nGridProductivity),dtype=float)

        iteration += 1
        if(iteration%20 == 0 or iteration == 1):
            print(" Iteration = ", iteration, ", Sup Diff = ", maxDifference)

    return (maxDifference, iteration, mValueFunction, mPolicyFunction)

if __name__ == '__main__':
    # - Start Timer - #
    t1=time.time()
    # - Call Main Function - #
    maxDiff, iterate, mValueF, mPolicyFunction = main_func()
    # - End Timer - #
    t2 = time.time()
    print(" Iteration = ", iterate, ", Sup Duff = ", maxDiff)
    print(" ")
    print(" My Check = ", mPolicyFunction[1000-1,3-1])
    print(" ")
    print("Elapse time = is ", t2-t1)

Output =  0.5627314338711378  Capital =  0.178198287392527  Consumption =  0.3845331464786108
 Iteration =  1 , Sup Diff =  0.05274159340733661
 Iteration =  20 , Sup Diff =  0.018703459886607154
 Iteration =  40 , Sup Diff =  0.006668541708074516
 Iteration =  60 , Sup Diff =  0.0023813118038720216
 Iteration =  80 , Sup Diff =  0.0008513397746897633
 Iteration =  100 , Sup Diff =  0.00030462324420166276
 Iteration =  120 , Sup Diff =  0.00010906950872124899
 Iteration =  140 , Sup Diff =  3.907108211809174e-05
 Iteration =  160 , Sup Diff =  1.4008644636520629e-05
 Iteration =  180 , Sup Diff =  5.026474537705994e-06
 Iteration =  200 , Sup Diff =  1.8035522479920019e-06
 Iteration =  220 , Sup Diff =  6.471316942313621e-07
 Iteration =  240 , Sup Diff =  2.3219657929729465e-07
 Iteration =  260 , Sup Diff =  8.331409406814316e-08
 Iteration =  280 , Sup Diff =  2.989378022899558e-08
 Iteration =  300 , Sup Diff =  1.0726129096028103e-08
 Iteration =  302 , Sup Duff =  9.681200952016

Let's run once more to eliminate compile time.

In [6]:

# - Start Timer - #
t1=time.time()
# - Call Main Function - #
maxDiff, iterate, mValueF, mPolicyFunction = main_func()
# - End Timer - #
t2 = time.time()
print(" Iteration = ", iterate, ", Sup Duff = ", maxDiff)
print(" ")
print(" My Check = ", mPolicyFunction[1000-1,3-1])
print(" ")
print("Elapse time = is ", t2-t1)

Output =  0.5627314338711378  Capital =  0.178198287392527  Consumption =  0.3845331464786108
 Iteration =  1 , Sup Diff =  0.05274159340733661
 Iteration =  20 , Sup Diff =  0.018703459886607154
 Iteration =  40 , Sup Diff =  0.006668541708074516
 Iteration =  60 , Sup Diff =  0.0023813118038720216
 Iteration =  80 , Sup Diff =  0.0008513397746897633
 Iteration =  100 , Sup Diff =  0.00030462324420166276
 Iteration =  120 , Sup Diff =  0.00010906950872124899
 Iteration =  140 , Sup Diff =  3.907108211809174e-05
 Iteration =  160 , Sup Diff =  1.4008644636520629e-05
 Iteration =  180 , Sup Diff =  5.026474537705994e-06
 Iteration =  200 , Sup Diff =  1.8035522479920019e-06
 Iteration =  220 , Sup Diff =  6.471316942313621e-07
 Iteration =  240 , Sup Diff =  2.3219657929729465e-07
 Iteration =  260 , Sup Diff =  8.331409406814316e-08
 Iteration =  280 , Sup Diff =  2.989378022899558e-08
 Iteration =  300 , Sup Diff =  1.0726129096028103e-08
 Iteration =  302 , Sup Duff =  9.681200952016

## Julia version

To run the Julia code you will need to download it from [here](https://github.com/jstac/julia_python_comparison) and place it in the present working directory. We remove the print statements and add a warm-up iteration to be consistent with the Python code.

In [12]:
!julia RBC_Julia.jl

First run (warm-up): 
  4.494059 seconds (2.62 M allocations: 1.337 GiB, 2.31% gc time)
Second run: 
  3.465944 seconds (1.84 k allocations: 1.211 GiB, 0.55% gc time)


## An Alternate Python Version

Incidentally, we would write the code a little differently.  For those who are interested, here's a version that's conforms more closely to PEP8 and other Python conventions.  

Parallelization of the Bellman maximization loop is included, which involves switching one `range` call to `prange`.  There is a small speed gain in the second run.  The gain is only small because the algorithm exploits several features of the model that save time in serial execution but make parallelization more challenging (due to different wait times across threads).

Variables are global for simplicity.  A natural next step would be to put them in a small class.  This would help with comparative statics.

In [16]:
from numba import prange, njit

In [17]:
β = 0.95
α = 1/3
A = np.array([0.9792, 0.9896, 1.0000, 1.0106, 1.0212])  # Productivity

# Transition matrix
π = np.array([[0.9727, 0.0273, 0.0000, 0.0000, 0.0000],
              [0.0041, 0.9806, 0.0153, 0.0000, 0.0000],
              [0.0000, 0.0082, 0.9837, 0.0082, 0.0000],
              [0.0000, 0.0000, 0.0153, 0.9806, 0.0041],
              [0.0000, 0.0000, 0.0000, 0.0273, 0.9727]])

# Steady state values
k_star = (α * β)**(1 / (1 - α))  # Steady state capital
y_star = k_star**α               # Steady state output
c_star = y_star - k_star         # Steady state consumption

# Set up capital grid
n = 35640                        # Number of grid points
k_grid = np.linspace(0.5 * k_star, 1.5 * k_star, n)

In [18]:
@njit(parallel=True)
def innerloop(k_grid, v, A, π, β):
    
    E_v = v @ π.T                    # Expected value function
    y_grid = np.outer(k_grid**α, A)  # Output grid
    
    v_new = np.empty_like(y_grid)
    σ = np.empty_like(y_grid)
    k_next = 0
    
    for a in prange(len(A)):
        
        for i in range(len(k_grid)):

            v_max = -np.inf
            k_prime = k_grid[0]

            for j in range(k_next, len(k_grid)):  
                y = y_grid[i, a]
                k = k_grid[j]
                c = y - k
                v_temp = (1 - β) * np.log(c) + β * E_v[j, a];

                if v_temp > v_max:
                    v_max = v_temp
                    k_prime = k_grid[j]
                    k_next = j
                else:
                    break 

            v_new[i, a] = v_max
            σ[i, a] = k_prime

    return v_new, σ

In [19]:
def solve_model(k_grid, 
                A, 
                π,
                α=1/3, 
                β=0.95,
                tol=1e-8,
                maxiter=1000):

    # Initialize values
    diff = 1e3
    i = 0
    v = np.zeros((len(k_grid), len(A)))

    while diff > tol and i < maxiter:       
            
        # Update value and policy functions for given productivity
        v_new, σ = innerloop(k_grid, v, A, π, β)

        diff = np.abs(v_new - v).max()
        v = v_new

        i += 1
        if(i % 20 == 0 or i == 1):
            print(f"Iteration {i}: diff = {diff}")
            
    print(f'Coverged in {i} iterations')
            
    return v_new, σ

In [20]:
%time solve_model(k_grid, A, π)

Iteration 1: diff = 0.05274159340733661
Iteration 20: diff = 0.018703461659313803
Iteration 40: diff = 0.006668550348659208
Iteration 60: diff = 0.0023813230210369962
Iteration 80: diff = 0.0008513486462362607
Iteration 100: diff = 0.00030462866646285836
Iteration 120: diff = 0.00010907234621893025
Iteration 140: diff = 3.907242347012119e-05
Iteration 160: diff = 1.4009761284849986e-05
Iteration 180: diff = 5.026883433956186e-06
Iteration 200: diff = 1.8037008289173428e-06
Iteration 220: diff = 6.471854268053079e-07
Iteration 240: diff = 2.3221595313316357e-07
Iteration 260: diff = 8.332106660180472e-08
Iteration 280: diff = 2.989628677951828e-08
Iteration 300: diff = 1.0727029597923377e-08
Coverged in 302 iterations
CPU times: user 8.36 s, sys: 157 ms, total: 8.51 s
Wall time: 3.71 s


(array([[-0.99728868, -0.98552368, -0.97409286, -0.9603671 , -0.95228169],
        [-0.99728731, -0.98552231, -0.97409149, -0.96036574, -0.95227998],
        [-0.99728594, -0.98552094, -0.97409012, -0.96036437, -0.95227828],
        ...,
        [-0.97049417, -0.95872916, -0.9472971 , -0.93356899, -0.9226772 ],
        [-0.97049371, -0.9587287 , -0.94729665, -0.93356853, -0.92267674],
        [-0.97049325, -0.95872825, -0.94729619, -0.93356808, -0.92267628]]),
 array([[0.13849505, 0.13996508, 0.14145011, 0.14299514, 0.2061863 ],
        [0.13849505, 0.13997008, 0.14145011, 0.14299514, 0.2061863 ],
        [0.13850006, 0.13997008, 0.14145511, 0.14300014, 0.2061863 ],
        ...,
        [0.19974118, 0.20186122, 0.20400626, 0.2061813 , 0.21277143],
        [0.19974118, 0.20186622, 0.20400626, 0.2061813 , 0.21277143],
        [0.19974118, 0.20186622, 0.20400626, 0.2061863 , 0.21277143]]))

In [21]:
%time solve_model(k_grid, A, π)

Iteration 1: diff = 0.05274159340733661
Iteration 20: diff = 0.018703461659313803
Iteration 40: diff = 0.006668550348659208
Iteration 60: diff = 0.0023813230210369962
Iteration 80: diff = 0.0008513486462362607
Iteration 100: diff = 0.00030462866646285836
Iteration 120: diff = 0.00010907234621893025
Iteration 140: diff = 3.907242347012119e-05
Iteration 160: diff = 1.4009761284849986e-05
Iteration 180: diff = 5.026883433956186e-06
Iteration 200: diff = 1.8037008289173428e-06
Iteration 220: diff = 6.471854268053079e-07
Iteration 240: diff = 2.3221595313316357e-07
Iteration 260: diff = 8.332106660180472e-08
Iteration 280: diff = 2.989628677951828e-08
Iteration 300: diff = 1.0727029597923377e-08
Coverged in 302 iterations
CPU times: user 7.51 s, sys: 153 ms, total: 7.66 s
Wall time: 2.88 s


(array([[-0.99728868, -0.98552368, -0.97409286, -0.9603671 , -0.95228169],
        [-0.99728731, -0.98552231, -0.97409149, -0.96036574, -0.95227998],
        [-0.99728594, -0.98552094, -0.97409012, -0.96036437, -0.95227828],
        ...,
        [-0.97049417, -0.95872916, -0.9472971 , -0.93356899, -0.9226772 ],
        [-0.97049371, -0.9587287 , -0.94729665, -0.93356853, -0.92267674],
        [-0.97049325, -0.95872825, -0.94729619, -0.93356808, -0.92267628]]),
 array([[0.13849505, 0.13996508, 0.14145011, 0.14299514, 0.2061863 ],
        [0.13849505, 0.13997008, 0.14145011, 0.14299514, 0.2061863 ],
        [0.13850006, 0.13997008, 0.14145511, 0.14300014, 0.2061863 ],
        ...,
        [0.19974118, 0.20186122, 0.20400626, 0.2061813 , 0.21277143],
        [0.19974118, 0.20186622, 0.20400626, 0.2061813 , 0.21277143],
        [0.19974118, 0.20186622, 0.20400626, 0.2061863 , 0.21277143]]))